# Playing with wawy things

In [ ]:
import numpy as np
from scipy.interpolate import griddata

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
D = (6, 6)

## Geneating grid for the simulation

In [ ]:
res = 20
xi = np.linspace(0, D[0], D[0]*res)
yi = np.linspace(0, D[1], D[1]*res)
X, Y = np.meshgrid(xi, yi)
coords = np.vstack((X.flatten(), Y.flatten())).T

In [ ]:
nr, nc = 1, 2
fig, axes = plt.subplots(nr, nc, figsize=(nr*D[0], nc*D[1]), dpi=120,
                         facecolor='black')
axes = axes.flatten()

for ax, C in zip(axes, [X, Y]):
    ax.axis('off')
    ax.imshow(C)

plt.show()

## Field of random particles

In [ ]:
N = 20000
x = np.random.uniform(X.min(), X.max(), size=N)
y = np.random.uniform(Y.min(), Y.max(), size=N)
points = np.vstack((x.flatten(), y.flatten())).T

In [ ]:
fig, ax = plt.subplots(figsize=D, dpi=120, facecolor='black')
ax.set_aspect('equal')
ax.axis('off')

ax.scatter(*points.T, color='white', marker='o', fc='none', s=10)

plt.show()

## Generating 2D waves

### Basic generation

In [ ]:
A = 1.0
c = 1.0

In [ ]:
def wave(t, X, Y, x0, y0, A, c, noise=True):
    r = np.sqrt((X-x0)**2 + (Y-y0)**2)
    wave = A * np.heaviside(t - r/c, 1) * np.cos(2*np.pi*(t - r/c))
    if noise:
        wave += np.random.normal(0, A/8, X.shape)
    return wave

In [ ]:
fig, ax = plt.subplots(figsize=D, dpi=120, facecolor='black')
ax.set_aspect('equal')
ax.axis('off')

Z = wave(t=10, X=X, Y=Y, x0=2, y0=1, A=1.0, c=2.5)
ax.contourf(X, Y, Z, levels=np.linspace(np.min(Z), np.max(Z), 50), cmap='RdBu')

plt.show()

### Interference of waves from symmetrically placed point sources

In [ ]:
a = 6
l = 3
xy1 = (a/2, a/2 + l)
xy2 = (a/2 - l*np.sqrt(3)/2, a/2 - l/2)
xy3 = (a/2 + l*np.sqrt(3)/2, a/2 - l/2)

In [ ]:
def polygon_vertices(a: float, l: float, n: int):
    '''
    Calculate the vertices of a regular polygon inscribed in a square.

    Parameters:
    a : float
        The side length of the square.
    l : float
        The distance from the center of the square to the top vertex of the polygon.
    n : int
        The number of sides of the polygon.

    Returns:
    vertices : list
        The coordinates of the vertices.
    '''
    if n < 2:
        raise ValueError("The number of sides `n` must be at least 2.")
    center_x, center_y = a/2, a/2
    vertices = []
    for i in range(n):
        angle = 2 * np.pi * i / n
        x = center_x + l * np.cos(angle)
        y = center_y + l * np.sin(angle)
        vertices.append((x, y))
    return vertices

In [ ]:
a, l, n = 6, 3, 7
vertices = polygon_vertices(a=a, l=l, n=n)

In [ ]:
fig, ax = plt.subplots(figsize=D, dpi=120, facecolor='black')
ax.set_aspect('equal')
ax.axis('off')

t = 15
Z = np.zeros_like(X)
for x0, y0 in vertices:
    Z += wave(t=t, X=X, Y=Y, x0=x0, y0=y0, A=1.0, c=1.0)
ax.contourf(X, Y, Z, levels=np.linspace(-n, n, 50), cmap='RdBu')

plt.show()

## Visualize waves with particles

In [ ]:
from scipy.interpolate import griddata

In [ ]:
def color_waves(Z_normed, cmap=None):
    '''
    Generate colormaps
    '''
    if cmap is None:
        cmap = cm.Blues_r
    colors = cmap(Z_normed)
    colors_a = colors.copy()
    colors_a[:, -1] = 0.05

    return colors, colors_a

In [ ]:
t = 30

In [ ]:
Z  = wave(t=t, X=X, Y=Y, x0=5, y0=4, A=1.0, c=1.5)
Z += wave(t=t, X=X, Y=Y, x0=17, y0=11, A=1.0, c=1.5)
Z_interp = griddata(coords, Z.flatten(), points, method='cubic')
Z_shift  = Z_interp - np.min(Z_interp)
Z_normed = Z_shift / np.max(Z_shift)

In [ ]:
fig, ax = plt.subplots(figsize=D, dpi=120, facecolor='black')
ax.set_aspect('equal')
ax.axis('off')

sizes = Z_normed * 16 + 4
ec, fc = color_waves(Z_normed)
ax.scatter(*points.T, fc=fc, ec=ec, lw=0.5, marker='o', s=sizes)

plt.show()

### Rocking particles dues to waves

In [ ]:
def rocking(Z, x, y, x0, y0):

    # Outward normed directions from the source
    directions = np.vstack((x - x0, y - y0)).T
    norms = np.sqrt((directions ** 2).sum(axis=-1))
    directions /= norms[:, np.newaxis]

    # Calculate radial distance from source
    r = np.sqrt((x - x0) ** 2 + (y - y0) ** 2)
    # Calculate phase velocity
    phase_velocity = -np.sin(2*np.pi*(t - r/c))

    # Compute the movements of the particles based on the wave amplitude
    disp = Z[:, np.newaxis] * directions * 0.1

    return x+disp[:, 0], y+disp[:, 1]

In [ ]:
nr, nc = 1, 2
fig, axes = plt.subplots(nr, nc, figsize=(nc*D[0], nr*D[1]), dpi=120,
                         facecolor='black', subplot_kw={'facecolor' : 'black'})
fig.subplots_adjust(wspace=0)
axes = axes.flatten()

sizes = Z_normed * 16 + 4
ec, fc = color_waves(Z_normed)
axes[0].scatter(x, y, fc=fc, ec=ec, lw=0.5, marker='o', s=sizes)

xnew, ynew = rocking(Z_interp, x, y, x0, y0)
axes[1].scatter(xnew, ynew, fc=fc, ec=ec, lw=0.5, marker='o', s=sizes)

plt.show()

## `Wave` and `WaveSimulation` classes

In [ ]:
class Wave:
    def __init__(self, t, A, c, x0, y0):
        self.t = t
        self.A = A
        self.c = c
        self.x0 = x0
        self.y0 = y0
        self.snr = None
        self.noise = False

    def step(self, dt):
        self.t += dt

    def add_noise(self, shape):
        noise = np.random.normal(0, self.A / self.snr, shape)
        return noise

    def calc(self, X, Y, noise=False, snr=None):
        r = np.sqrt((X-self.x0)**2 + (Y-self.y0)**2)
        wave  = self.A * np.cos(2*np.pi*(self.t - r/self.c))
        wave *= np.heaviside(self.t - r/self.c, 1)
        if noise:
            self.noise = True
            if snr is None:
                self.snr = 10
            else:
                self.snr = snr
            wave += self.add_noise(wave.shape)
        return wave

In [ ]:
test_wave = Wave(t=0, A=0.0, c=1.0, x0=0, y0=0)
Z = test_wave.calc(X, Y, noise=False)

In [ ]:
fig, ax = plt.subplots(figsize=D, dpi=120, facecolor='black')
ax.set_aspect('equal')
ax.axis('off')

ax.contourf(X, Y, Z, cmap='RdBu', vmin=-test_wave.A, vmax=test_wave.A)

plt.show()

In [ ]:
class WaveSimulation():
    def __init__(self, *,
                 d_size : tuple, res : int, num_part : int):

        self.D = d_size
        self.res = res
        self.N = num_part

        self.X, self.Y = self.__init_grid()
        self.particles = self.__init_particles()

        self.components = []
        self.wave = np.zeros(self.X.shape)

    def __init_grid(self):
        xi = np.linspace(0, self.D[0], self.D[0]*self.res)
        yi = np.linspace(0, self.D[1], self.D[1]*self.res)
        return np.meshgrid(xi, yi)

    def __init_particles(self):
        return np.random.uniform([self.X.min(), self.Y.min()],
                                 [self.X.max(), self.Y.max()],
                                 size=(self.N, 2))

    def add_wave(self, *, t, A, c, x0, y0, noise=False, snr=5):
        wave = Wave(t, A, c, x0, y0)
        self.components.append(wave)
        self.wave += wave.calc(self.X, self.Y, noise, snr)

    def interpolate(self):
        if not self.components:
            return
        points = np.vstack((self.X.flatten(), self.Y.flatten())).T
        values = self.wave.flatten()
        interp = griddata(points, values, self.particles, method='cubic')
        shift  = interp - np.min(interp)
        normed = shift / np.max(shift) if np.max(shift) != 0 else np.zeros_like(shift)
        return normed

    def displacement(self):
        if not self.components:
            return
        particles_disp = np.zeros_like(self.particles)
        for wave in self.components:
            x0, y0, c, t = wave.x0, wave.y0, wave.c, wave.t
            directions = self.particles - np.array([x0, y0])
            norms = np.sqrt((directions ** 2).sum(axis=-1))
            directions /= norms[:, np.newaxis]
            r = np.sqrt(((self.particles - np.array([x0, y0])) ** 2).sum(axis=-1))
            phase_velocity = -np.sin(2*np.pi*(t - r/c))

            # Calculate wave at the particle locations
            wave_at_particles = wave.calc(*self.particles.T)
            particles_disp += (wave_at_particles[:, np.newaxis]
                               * directions
                               * phase_velocity[:, np.newaxis]
                               * 0.15)
        return particles_disp

    def step(self, *, dt):
        if not self.components:
            return
        new_wave = np.zeros(self.X.shape)
        for wave in self.components:
            wave.step(dt)
            new_wave += wave.calc(self.X, self.Y, wave.noise, wave.snr)
        self.wave = new_wave

In [ ]:
sim = WaveSimulation(d_size=(6, 6), res=20, num_part=12000)
sim.add_wave(t=30, A=1.0, c=1.5, x0=5, y0=4, noise=True)
sim.add_wave(t=30, A=1.0, c=1.5, x0=17, y0=11, noise=False)

Z = sim.wave
Z_interp = sim.interpolate()
part_disp = sim.particles + sim.displacement()

In [ ]:
nr, nc = 1, 2
fig, axes = plt.subplots(nr, nc, figsize=(nc*6, nr*6), dpi=120, facecolor='black')
fig.subplots_adjust(wspace=0.0)

for ax in axes.flatten():
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_xlim(np.min(sim.X), np.max(sim.X))
    ax.set_ylim(np.min(sim.Y), np.max(sim.Y))

axes[0].contourf(sim.X, sim.Y, Z, cmap='RdBu_r')

sizes = Z_interp * 16 + 4
ec, fc = color_waves(Z_interp, cmap=cm.Blues_r)
axes[1].scatter(*part_disp.T, fc=fc, ec=ec, lw=0.5, marker='o', s=sizes)

plt.show()

## Animate surface waves

In [ ]:
def visualize_field(sim, total_frames, fps=60, dt=0.01,
                    name='WaveSimulationField.mp4', cmap='RdBu'):
    fig, ax = plt.subplots(figsize=sim.D, dpi=120,
                           facecolor='black', subplot_kw={'facecolor': 'black'})
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    ax.set_aspect('equal')
    ax.axis('off')

    lims = np.sum([wave.A for wave in sim.components])

    def animate(i):
        sim.step(dt=dt)
    
        ax.clear()
        ax.contourf(sim.X, sim.Y, sim.wave, cmap=cmap,
                    levels=np.linspace(-lims, lims, 50),
                    vmin=-lims, vmax=lims)
        ax.set_xlim(np.min(sim.X)-0.1*sim.D[0], np.max(sim.X)+0.1*sim.D[0])
        ax.set_ylim(np.min(sim.Y)-0.1*sim.D[1], np.max(sim.Y)+0.1*sim.D[1])

    ani = animation.FuncAnimation(fig, animate, frames=total_frames)
    plt.show()

    # Save animation
    ani.save(name, fps=fps, writer='ffmpeg')
    # or save as gif
    # ani.save('wave_animation.gif', writer='imagemagick')

In [ ]:
%%time
sim = WaveSimulation(d_size=(6, 6), res=100, num_part=1)
sim.add_wave(t=30, A=1.0, c=1.5, x0=5, y0=1, noise=False)
sim.add_wave(t=30, A=1.0, c=1.5, x0=17, y0=11, noise=False)
visualize_field(sim, total_frames=200, fps=30, dt=0.01)

In [ ]:
def visualize_particles(sim, total_frames, fps=60, dt=0.01,
                        name='WaveSimulationParticles.mp4'):
    fig, ax = plt.subplots(figsize=sim.D, dpi=120,
                           facecolor='black', subplot_kw={'facecolor': 'black'})
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    ax.set_aspect('equal')
    ax.axis('off')

    def animate(i):
        sim.step(dt=dt)
        
        #Z = sim.wave
        Z_interp = sim.interpolate()
        part_disp = sim.particles + sim.displacement()
    
        sizes = Z_interp * 16 + 4
        ec, fc = color_waves(Z_interp)
    
        ax.clear()
        ax.scatter(*part_disp.T, fc=fc, ec=ec, lw=0.5, marker='o', s=sizes)
        ax.set_xlim(np.min(sim.X)-0.1*sim.D[0], np.max(sim.X)+0.1*sim.D[0])
        ax.set_ylim(np.min(sim.Y)-0.1*sim.D[1], np.max(sim.Y)+0.1*sim.D[1])

    ani = animation.FuncAnimation(fig, animate, frames=total_frames)
    plt.show()

    # Save animation
    ani.save(name, fps=fps, writer='ffmpeg')
    # or save as gif
    # ani.save('wave_animation.gif', writer='imagemagick')

In [ ]:
%%time
sim = WaveSimulation(d_size=(6, 6), res=20, num_part=12000)
sim.add_wave(t=30, A=1.0, c=1.5, x0=5, y0=1, noise=True)
sim.add_wave(t=30, A=1.0, c=1.5, x0=17, y0=11, noise=False)
visualize_particles(sim, total_frames=200, fps=30, dt=0.01)

In [ ]:
%%time
sim = WaveSimulation(d_size=(6, 6), res=20, num_part=12000)
sim.add_wave(t=4, A=1.0, c=2.0, x0=1.5, y0=1.5, noise=False)
visualize_particles(sim, total_frames=300, fps=60, dt=0.01, name='SingleWaveLoop.mp4')

In [ ]:
%%time
a, l, n = 6, 3, 7
vertices = polygon_vertices(a=a, l=l, n=n)

sim = WaveSimulation(d_size=(a, a), res=40, num_part=1)
for x0, y0 in vertices:
    sim.add_wave(t=30, A=1.0, c=1.0, x0=x0, y0=y0, noise=False)
visualize_field(sim, total_frames=200, fps=30, dt=0.01,
                name='PolygonField.mp4', cmap=cm.coolwarm)

## Just a couple little test

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=120, facecolor='black')
ax.set_aspect('equal')
ax.axis('off')
ax.set_xlim(np.min(X), np.max(X))
ax.set_ylim(np.min(Y), np.max(Y))

t = 15
x0, y0 = 1, 1
Z1 = wave(t=t, X=X, Y=Y, x0=x0, y0=y0, A=1.0, c=2.0, noise=False)
Z2 = wave(t=t+1.0, X=X, Y=Y, x0=x0, y0=y0, A=1.0, c=2.0, noise=False)
ax.contourf(X, Y, Z1-Z2, cmap='RdBu', vmin=-1, vmax=1)

plt.show()